In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from torch.autograd import Variable 
from torchvision import transforms 
from data_loader import build_vocab 
from model import EncoderCNN, DecoderRNN
from model import ResNet, ResidualBlock
from PIL import Image


embed_size = 256
hidden_size = 512 
num_layers = 1 
decoder_path = './models/3object_color/decoder-10-150.pkl'
encoder_path = './models/3object_color/encoder-10-150.pkl'


def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def load_image(image_path, transform=None):
    image = Image.open(image_path).convert('RGB')
    image = image.resize([64, 64], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.033, 0.032, 0.033), 
                         (0.027, 0.027, 0.027))])



vocab_path = './data/vocab_3object_color.pkl'
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)
len_vocab = vocab.idx

# Build Models
encoder = ResNet(ResidualBlock, [3, 3, 3], len_vocab)
encoder.eval()  # evaluation mode (BN uses moving mean/variance)
decoder = DecoderRNN(len_vocab, hidden_size, 
                     len(vocab), num_layers)


# Load the trained model parameters
encoder.load_state_dict(torch.load(encoder_path))
decoder.load_state_dict(torch.load(decoder_path))

# If use gpu
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

    

In [2]:
test_dir = 'data/3object_test/bitmap/'
cap_dir = 'data/3object_test/caption/'
test_list = os.listdir(test_dir)
cnt = 0 

shape_match = 0
location_match = 0
radius_match = 0 
rect_x_match = 0 
rect_y_match = 0 
length_match = 0
for fname in test_list: 
    #cnt+=1 
    if cnt>2:
        break;
    test_path = test_dir + fname
    test_image = load_image(test_path, transform)
    image_tensor = to_var(test_image)
    
    # Generate caption from image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    ids_arr = []
    for element in sampled_ids: 
        temp = element.cpu().data.numpy()
        ids_arr.append(int(temp))
    print(fname)

    # Decode word_ids to words
    sampled_caption = []
    for word_id in ids_arr:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    in_caption = sampled_caption[1:-1]
    print(in_caption)
    
    #read target caption
    cap_target = cap_dir+fname
    cap_path = cap_target.split('.')[0]+'.svg'
    trg_caption = [] 
    with open(cap_path, 'r') as f:
        target = f.readline().split()
        for word in target:
            trg_caption.append(word)
    print(trg_caption)
    print('-------------------------------------------------------------------')
    
    
    if(len(in_caption) == len(trg_caption)):        
        length_match += 1
        
print(length_match) 


9908.png
['circle', '2', '9', '110', 'red', 'circle', '6', '2', '50', 'purple', 'rect', '4', '7', '110', '110', 'red']
['rect', '4', '1', '90', '90', 'red', 'rect', '8', '10', '70', '70', 'orange', 'circle', '2', '6', '80', 'red']
-------------------------------------------------------------------
9759.png
['circle', '7', '4', '110', 'red', 'circle', '2', '4', '110', 'red']
['circle', '9', '6', '60', 'orange', 'circle', '3', '3', '110', 'lime', 'rect', '6', '7', '60', '60', 'red']
-------------------------------------------------------------------
9903.png
['rect', '10', '9', '110', '110', 'red', 'rect', '10', '5', '110', '110', 'purple', 'circle', '7', '10', '110', 'red']
['rect', '10', '6', '120', '120', 'blue', 'rect', '9', '2', '40', '40', 'deep_pink', 'circle', '6', '10', '120', 'deep_pink']
-------------------------------------------------------------------
9980.png
['rect', '10', '2', '30', '30', 'red', 'rect', '3', '3', '30', '30', 'red']
['rect', '9', '3', '30', '30', 'lime', 

In [3]:
print(length_match/len(test_list))

0.82
